In [1]:
import pandas as pd

# Crear DataFrame con las coordenadas específicas proporcionadas
torres_df = pd.DataFrame({
    'ID_Torre': ['TORRE_001', 'TORRE_002', 'TORRE_003', 'TORRE_004', 
                'TORRE_005', 'TORRE_006', 'TORRE_007'],
    'Nombre': ['Torre 1', 'Torre 2', 'Torre 3', 'Torre 4', 
              'Torre 5', 'Torre 6', 'Torre 7'],
    'Latitud': [6.642631, 6.858107, 6.841745, 6.356094, 
               6.710830, 6.634446, 6.214130],
    'Longitud': [-71.814700, -71.902591, -71.391727, -70.825931,
                -70.666629, -71.314822, -71.946536]
})

# Mostrar el DataFrame creado
print("DataFrame creado con las coordenadas específicas:")
print(torres_df)

# Guardar el CSV
ruta_torres = "../data/03_external/ubicacion_torres.csv"
torres_df.to_csv(ruta_torres, index=False)
print(f"\n✅ Archivo guardado en: {ruta_torres}")
print(f"📍 Total de torres: {len(torres_df)}")

DataFrame creado con las coordenadas específicas:
    ID_Torre   Nombre   Latitud   Longitud
0  TORRE_001  Torre 1  6.642631 -71.814700
1  TORRE_002  Torre 2  6.858107 -71.902591
2  TORRE_003  Torre 3  6.841745 -71.391727
3  TORRE_004  Torre 4  6.356094 -70.825931
4  TORRE_005  Torre 5  6.710830 -70.666629
5  TORRE_006  Torre 6  6.634446 -71.314822
6  TORRE_007  Torre 7  6.214130 -71.946536

✅ Archivo guardado en: ../data/03_external/ubicacion_torres.csv
📍 Total de torres: 7


In [2]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point

# 1. Cargar los datos de las torres (necesitarás crear este archivo CSV)
# Asegúrate de tener un archivo CSV en: data/03_external/ubicacion_torres.csv
# Con las columnas: ID_Torre, Latitud, Longitud
try:
    torres_df = pd.read_csv("../data/03_external/ubicacion_torres.csv")
    print("✅ Archivo de torres cargado con éxito.")
except FileNotFoundError:
    print("⚠️ Archivo no encontrado. Creando un DataFrame de ejemplo.")
    # DataFrame de ejemplo si no tienes el archivo real todavía
    torres_df = pd.DataFrame({
        'ID_Torre': ['T-101', 'T-102', 'T-103', 'T-104'],
        'Latitud': [7.08, 7.10, 7.12, 7.05],
        'Longitud': [-72.25, -72.28, -72.30, -72.20]
    })

# 2. Extraer los valores del raster para cada coordenada de torre
coords = [(x, y) for x, y in zip(torres_df.Longitud, torres_df.Latitud)]

# Apuntamos a nuestro raster clasificado y reproyectado
ruta_raster_final = "../data/02_processed/amenaza_arauca_clasificado_wgs84.tif"

with rasterio.open(ruta_raster_final) as src:
    # 'sample' es un generador, lo convertimos a una lista
    valores_raster = [val[0] for val in src.sample(coords)]

# 3. Añadir los resultados al DataFrame
torres_df['Amenaza_Valor'] = valores_raster

# Mapear los valores numéricos a etiquetas legibles
mapa_amenaza_labels = {1: 'Baja', 2: 'Media', 3: 'Alta', 0: 'Sin Datos'}
torres_df['Amenaza_Nivel'] = torres_df['Amenaza_Valor'].map(mapa_amenaza_labels)

# 4. Mostrar el resultado final
print("\nResultado del cruce de información:")
print(torres_df)

# Guardar el resultado para uso futuro
ruta_salida_final = "../data/02_processed/torres_con_amenaza.csv"
torres_df.to_csv(ruta_salida_final, index=False)
print(f"\n✅ Datos de torres con nivel de amenaza guardados en: {ruta_salida_final}")

✅ Archivo de torres cargado con éxito.

Resultado del cruce de información:
    ID_Torre   Nombre   Latitud   Longitud  Amenaza_Valor Amenaza_Nivel
0  TORRE_001  Torre 1  6.642631 -71.814700              2         Media
1  TORRE_002  Torre 2  6.858107 -71.902591              1          Baja
2  TORRE_003  Torre 3  6.841745 -71.391727              1          Baja
3  TORRE_004  Torre 4  6.356094 -70.825931              3          Alta
4  TORRE_005  Torre 5  6.710830 -70.666629              1          Baja
5  TORRE_006  Torre 6  6.634446 -71.314822              2         Media
6  TORRE_007  Torre 7  6.214130 -71.946536              3          Alta

✅ Datos de torres con nivel de amenaza guardados en: ../data/02_processed/torres_con_amenaza.csv
